Create a nanotube. 

In [ ]:
from ase.build import nanotube
from ase import Atoms
from ase.io import write
import numpy as np

# Parameters for the nanotube
n = 6  # Chirality n
m = 6  # Chirality m
length = 6
# Length in unit cells
bond_length = 1.42  # Bond length for carbon-carbon

# Create the nanotube
cnt = nanotube(n, m, length=length)
write('nanotube.xyz', cnt)

Creating capped nanotubes: decent idea and seems to be working but much safer to just create a simple nanotube and then place hydrogens manually in avogadro or something like that. 

In [1]:
from ase.build import nanotube
from ase import Atoms
from ase.io import write
import numpy as np

# Parameters for the nanotube
n = 6  # Chirality n
m = 6  # Chirality m
length = 6
# Length in unit cells
bond_length = 1.42  # Bond length for carbon-carbon

# Create the nanotube
cnt = nanotube(n, m, length=length)

# Get positions of carbon atoms
positions = cnt.get_positions()

# Identify the carbon atoms at the ends
z_min = np.min(positions[:, 2])
z_max = np.max(positions[:, 2])
tolerance = 0.1  # Tolerance to include all end atoms

end_atoms_min = positions[np.where(np.abs(positions[:, 2] - z_min) < tolerance)]
end_atoms_max = positions[np.where(np.abs(positions[:, 2] - z_max) < tolerance)]

# Function to add hydrogen atoms to the end atoms
def add_hydrogens(end_atoms, direction):
    hydrogens = []
    for atom in end_atoms:
        hydrogen_position = atom + direction * np.array([0, 0, bond_length])
        hydrogens.append(hydrogen_position)
    return hydrogens

# Add hydrogen atoms at both ends
hydrogens_min = add_hydrogens(end_atoms_min, -1)
hydrogens_max = add_hydrogens(end_atoms_max, 1)

# Create hydrogen atoms
hydrogen_atoms = Atoms('H' * (len(hydrogens_min) + len(hydrogens_max)),
                       positions=hydrogens_min + hydrogens_max)

# Combine the nanotube and hydrogen atoms
capped_cnt = cnt + hydrogen_atoms

# Save the structure to an XYZ file
write('capped_nanotube_2.xyz', capped_cnt)


creating packmol input for filling a single nanotube with water. separated from nanotube creating script, cause I have more trust in adding hydrogens manually. 

In [7]:
import numpy as np
#input xyz paths 
nanotube_path = "nanotube.xyz"
water_path = output_path = "water.xyz"
#new xyz file to be created 
output_path = "packed_system.xyz"
#packmol input to be created
packmol_input_path = "input.in"
number_of_water_molecules = 1
def read_xyz(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        atom_count = int(lines[0])
        atoms = []

        for line in lines[2:2 + atom_count]:
            parts = line.split()
            atom_type = parts[0]
            x, y, z = map(float, parts[1:])
            atoms.append((atom_type, x, y, z))

    return atoms

def calculate_center_of_nanotube(atoms):
    coords = np.array([(x, y, z) for _, x, y, z in atoms])
    center = np.mean(coords, axis=0)
    return center

def generate_packmol_input(center, output_path):
    with open(output_path, 'w') as file:
        file.write("tolerance 2.0\n")
        file.write("filetype xyz\n")
        file.write(f"output {output_path}\n\n")
        
        # Add nanotube structure
        file.write(f"structure {nanotube_path}\n")
        file.write(f"  number {number_of_water_molecules}\n")
        file.write(f"  inside box {center[0]-10} {center[1]-10} {center[2]-10} {center[0]+10} {center[1]+10} {center[2]+10}\n")
        file.write("end structure\n\n")

        # Add water molecules structure
        file.write(f"structure {water_path}\n")
        file.write("  number 20\n")
        file.write(f"  inside box {center[0]-1} {center[1]-1} {center[2]-5} {center[0]+1} {center[1]+1} {center[2]+5}\n")
        file.write("end structure\n")



# Read nanotube coordinates and calculate center
nanotube_atoms = read_xyz(nanotube_path)
center_of_nanotube = calculate_center_of_nanotube(nanotube_atoms)

# Generate Packmol input file
generate_packmol_input(center_of_nanotube, packmol_input_path)

print(f"Center of nanotube: {center_of_nanotube}")
print(f"Packmol input file '{packmol_input_path}' generated successfully.")


Center of nanotube: [-2.39523899 -0.90398254  9.84808   ]
Packmol input file 'input.in' generated successfully.


In [ ]:
.xyz"
water_path = output_path = "water.xyz"
#new xyz file to be created 
output_path = "packed_system.xyz"
#packmol input to be created
packmol_input_path = "input.in"
number_of_water_molecules = 1
def read_xyz(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        atom_count = int(lines[0])
        atoms = []

        for line in lines[2:2 + atom_count]:
            parts = line.split()
            atom_type = parts[0]
            x, y, z = map(float, parts[1:])
            atoms.append((atom_type, x, y, z))

    return atoms

def calculate_center_of_nanotube(atoms):
    coords = np.array([(x, y, z) for _, x, y, z in atoms])
    center = np.mean(coords, axis=0)
    return center

def generate_packmol_input(center, output_path):
    with open(output_path, 'w') as file:
        file.write("tolerance 2.0\n")
        file.write("filetype xyz\n")
        file.write(f"output {output_path}\n\n")
        
        # Add nanotube structure
        file.write(f"structure {nanotube_path}\n")
        file.write(f"  number {number_of_water_molecules}\n")
        file.write(f"  inside box {center[0]-10} {center[1]-10} {center[2]-10} {center[0]+10} {center[1]+10} {center[2]+10}\n")
        file.write("end structure\n\n")

        # Add water molecules structure
        file.write(f"structure {water_path}\n")
        file.write("  number 20\n")
        file.write(f"  inside box {center[0]-1} {center[1]-1} {center[2]-5} {center[0]+1} {center[1]+1} {center[2]+5}\n")
        file.write("end structure\n")



# Read nanotube coordinates and calculate center
nanotube_atoms = read_xyz(nanotube_path)
center_of_nanotube = calculate_center_of_nanotube(nanotube_atoms)

# Generate Packmol input file
generate_packmol_input(center_of_nanotube, packmol_input_path)

print(f"Center of nanotube: {center_of_nanotube}")
print(f"Packmol input file '{packmol_input_path}' generated successfully.")


creating multiple packmol inputs for nanotubes of different diameters and lengths

In [16]:
import numpy as np
diameters = [3, 6, 12]  # Example list of diameters of nanotubes
lengths = [20, 30, 40] # Example list of lengths of nanotubes
# Generalized file paths for nanotube and water XYZ files
nanotube_base_path = "capped_nanotube/nanotube_{d}/{l}/nanotube.xyz"
water_base_path = "water.xyz"  # Assuming the same water file is used for all configurations

# Output file paths
output_base_path = "capped_nanotube/nanotube_{d}/{l}/packed_system_{d}_{l}.xyz"
packmol_input_base_path = "capped_nanotube/nanotube_{d}/{l}/input_{d}_{l}.in"

# Number of water molecules based on nanotube diameter - that was just trial and error obtained expression 
def calculate_number_of_water_molecules(diameter):
    return 3 * (diameter / 3) ** 2

# Read an XYZ file to extract atom coordinates
def read_xyz(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        atom_count = int(lines[0])
        atoms = []

        for line in lines[2:2 + atom_count]:
            parts = line.split()
            atom_type = parts[0]
            x, y, z = map(float, parts[1:])
            atoms.append((atom_type, x, y, z))

    return atoms

# Calculate the center of a nanotube based on its atom coordinates
def calculate_center_of_nanotube(atoms):
    coords = np.array([(x, y, z) for _, x, y, z in atoms])
    center = np.mean(coords, axis=0)
    return center

# Generate the Packmol input file
def generate_packmol_input(water, center, output_path, nanotube_xyz_path):
    with open(output_path, 'w') as file:
        file.write("tolerance 2.0\n")
        file.write("filetype xyz\n")
        file.write(f"output nanotube_with_water_{water}.xyz\n\n")
        
        # Add nanotube structure
        file.write(f"structure {nanotube_xyz_path}\n")
        file.write("  fixed 0. 0. 0. 0. 0. 0.\n")
        file.write("end structure\n\n")

        # Add water molecules structure
        file.write(f"structure {water_base_path}\n")
        file.write(f"  number {int(water)}\n")
        x_center, y_center, z_center = center
        file.write(f"  inside box {x_center-0.5} {y_center-0.5} {z_center-5} {x_center+0.5} {y_center+0.5} {z_center+5}\n")
        file.write("end structure\n")

# Generate Packmol input and output files for various nanotube dimensions
for d in diameters:  
    for l in lengths:  
        water = calculate_number_of_water_molecules(d)  # Calculate water molecule count based on diameter
        nanotube_xyz_path = nanotube_base_path.format(d=d, l=l)
        packmol_input_path = packmol_input_base_path.format(d=d, l=l)
        
        # Read nanotube coordinates and calculate center
        nanotube_atoms = read_xyz(nanotube_xyz_path)
        center_of_nanotube = calculate_center_of_nanotube(nanotube_atoms)

        # Generate Packmol input file
        generate_packmol_input(water, center_of_nanotube, packmol_input_path, nanotube_xyz_path)

        print(f"Center of nanotube (d={d}, l={l}): {center_of_nanotube}")
        print(f"Packmol input file '{packmol_input_path}' generated successfully.")
        print(f"Number of water molecules: {water}")


Center of nanotube: [1.869045   3.23728149 9.85712579]
Packmol input file 'capped_nanotube/nanotube_3/20/input.in' generated successfully.
3.0
Center of nanotube: [ 1.869045    3.23728154 14.78568963]
Packmol input file 'capped_nanotube/nanotube_3/30/input.in' generated successfully.
3.0
Center of nanotube: [ 1.869045    3.23728157 19.71425257]
Packmol input file 'capped_nanotube/nanotube_3/40/input.in' generated successfully.
3.0
Center of nanotube: [-2.39523899 -0.90398254  9.84808   ]
Packmol input file 'capped_nanotube/nanotube_6/20/input.in' generated successfully.
12.0
Center of nanotube: [ 2.87933247  4.98715    14.77212   ]
Packmol input file 'capped_nanotube/nanotube_6/30/input.in' generated successfully.
12.0
Center of nanotube: [ 2.87933248  4.98715    19.69616   ]
Packmol input file 'capped_nanotube/nanotube_6/40/input.in' generated successfully.
12.0
Center of nanotube: [4.91117353 8.50640118 9.84575684]
Packmol input file 'capped_nanotube/nanotube_12/20/input.in' generate

also creates nanotubes, but easier to set length

In [ ]:
from ase import Atoms
from ase.build import nanotube
from ase.io import write
import numpy as np

# Generalized variables
chiral_indices = (6, 6)  # Chiral indices of the nanotube
nanotube_length = 40  # Length of the nanotube in Angstroms
unit_length = 2.46  # Unit length for nanotube periodicity
vacuum_space = 5.0  # Vacuum space around the nanotube in all directions
box_padding = 5.0  # Padding for the box size in z-direction
water_molecule_count = 30  # Number of water molecules
output_nanotube_file = "carbon_nanotube_periodic.xyz"  # Output nanotube XYZ file path
output_packmol_file = "packmol_input.inp"  # Output Packmol input file path
water_file_path = "water_molecules.xyz"  # Path to the water molecules XYZ file

# Create the nanotube
cnt = nanotube(n=chiral_indices[0], m=chiral_indices[1], length=int(nanotube_length/unit_length))

# Define the box size slightly larger than the nanotube length
box_length = cnt.cell[2, 2] + box_padding  # Adding padding in the z-direction
box_size = [cnt.cell[0, 0], cnt.cell[1, 1], box_length]

# Set the cell size for the nanotube
cnt.set_cell(box_size)

# Center the nanotube in the box with vacuum space
cnt.center(vacuum=vacuum_space, axis=(0, 1, 2))

# Apply periodic boundary conditions in all directions
cnt.set_pbc([True, True, True])

# Write the nanotube structure to an XYZ file
write(output_nanotube_file, cnt)

# Read atoms from the nanotube and store their coordinates
def read_atoms_from_nanotube(cnt):
    atoms = []
    for atom in cnt:
        atom_type = atom.symbol
        x, y, z = atom.position
        atoms.append((atom_type, x, y, z))
    return atoms

# Calculate the center of the nanotube
def calculate_center_of_nanotube(atoms):
    coords = np.array([(x, y, z) for _, x, y, z in atoms])
    center = np.mean(coords, axis=0)
    return center

# Generate the Packmol input file to insert water molecules into the nanotube
def generate_packmol_input(water, center, output_path, nanotube_file, water_file):
    with open(output_path, 'w') as file:
        file.write("tolerance 2.0\n")
        file.write("filetype xyz\n")
        file.write(f"output {nanotube_file}\n\n")
        
        # Add nanotube structure
        file.write(f"structure {nanotube_file}\n")
        file.write("  fixed 0. 0. 0. 0. 0. 0.\n")
        file.write("end structure\n\n")

        # Add water molecules structure
        file.write(f"structure {water_file}\n")
        file.write(f"  number {int(water)}\n")
        x_center, y_center, z_center = center
        # Define the box where water molecules will be placed inside the nanotube
        file.write(f"  inside box {x_center-0.5} {y_center-0.5} {z_center-5} {x_center+0.5} {y_center+0.5} {z_center+5}\n")
        file.write("end structure\n")

# Main execution
atoms = read_atoms_from_nanotube(cnt)
center = calculate_center_of_nanotube(atoms)
generate_packmol_input(water=water_molecule_count, center=center, output_path=output_packmol_file, nanotube_file=output_nanotube_file, water_file=water_file_path)
